In [2]:
import csv
import json
import os
import glob

# 디렉토리 경로 설정
json_dir = './Dataset/json_adjusted/'
csv_file = './datasets.csv'

# CSV 파일 생성
csv_columns = ['file_name', 'width', 'height', 'label', 'xmin', 'ymin', 'xmax', 'ymax']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()

    # 각 JSON 파일 처리
    json_files = glob.glob(os.path.join(json_dir, '*.json'))
    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        for image_info in data['images']:
            file_name = image_info['file_name']
            width = image_info['width']
            height = image_info['height']
            
            for annotation in data['annotations']:
                bbox = annotation['bbox']
                if bbox is not None:
                    row = {
                        'file_name': file_name,
                        'width': width,
                        'height': height,
                        'label': annotation['category_name'],
                        'x_min': bbox[0][0],
                        'y_min': bbox[0][1],
                        'x_max': bbox[1][0],
                        'y_max': bbox[1][1]
                    }
                    writer.writerow(row)

print("CSV 파일이 생성되었습니다.")

CSV 파일이 생성되었습니다.


In [1]:
import json
import os
import glob
import csv

def scale_bbox(bbox, x_scale, y_scale):
    x_min = bbox[0][0] * x_scale
    y_min = bbox[0][1] * y_scale + 90
    x_max = bbox[1][0] * x_scale
    y_max = bbox[1][1] * y_scale + 90
    return [x_min, y_min, x_max, y_max]

def json_to_csv(json_dirs, csv_file, x_scale, y_scale):
    csv_columns = ['file_name', 'width', 'height', 'label', 'x_min', 'y_min', 'x_max', 'y_max']
    with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()

        for json_dir in json_dirs:
            json_files = glob.glob(os.path.join(json_dir, '*.json'))
            for json_file in json_files:
                with open(json_file, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    
                    for image_info, annotation_info in zip(data['images'], data['annotations']):
                        bbox = annotation_info['bbox']
                        scaled_bbox = scale_bbox(bbox, x_scale, y_scale)
                        
                        row = {
                            'file_name': image_info['file_name'],
                            'width': 416,
                            'height': 416,
                            'label': annotation_info['category_name'],
                            'x_min': scaled_bbox[0],
                            'y_min': scaled_bbox[1],
                            'x_max': scaled_bbox[2],
                            'y_max': scaled_bbox[3]
                        }
                        writer.writerow(row)

# JSON 파일 디렉토리 경로들
json_dirs = ['./Dataset/json']
# 출력 CSV 파일 경로
csv_file = 'output.csv'

# 스케일 비율
x_scale = 0.21666666666666
y_scale = 0.21666666666666

# JSON 파일을 CSV로 변환
json_to_csv(json_dirs, csv_file, x_scale, y_scale)